In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL for EuroMillions draw history
url = "https://www.beatlottery.co.uk/euromillions/draw-history"

# List to hold all draws data
draws_data = []

# Function to extract draw data
def extract_draw_data():
    print("Extracting draws from the EuroMillions draw history page...")
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        # Locate the table containing the draw history
        table = soup.find('table')

        if table:
            rows = table.find_all('tr')[1:]  # Skip header row
            print(f"Found {len(rows)} rows in the table.")  # Debug: Print number of rows

            for row in rows:
                cells = row.find_all('td')
                if len(cells) >= 5:  # Ensure there are enough cells
                    draw_date = cells[0].get_text(strip=True)  # Extracting Draw Date
                    balls_raw = cells[2].get_text(strip=True) + " " + cells[3].get_text(strip=True)  # Main and Lucky Numbers

                    # Split main balls and lucky stars
                    main_balls = balls_raw.split("LUCKY STARS")[0].strip()
                    lucky_stars = balls_raw.split("LUCKY STARS")[1].strip() if "LUCKY STARS" in balls_raw else ""

                    # Format main balls and lucky stars with commas
                    main_balls_formatted = ", ".join(main_balls[i:i + 2] for i in range(0, len(main_balls), 2))
                    lucky_stars_formatted = ", ".join(lucky_stars[i:i + 2] for i in range(0, len(lucky_stars), 2))

                    # Append the data as a dictionary
                    draws_data.append({
                        'Draw Date': draw_date,
                        'Main Balls': main_balls_formatted,
                        'Lucky Stars': lucky_stars_formatted
                    })
                else:
                    print("Row does not have enough cells:", cells)  # Debug: Print row content
        else:
            print("No table found on the page.")
    else:
        print(f"Failed to retrieve data: {response.status_code}")

# Extract data
extract_draw_data()

# Create a DataFrame from the collected data
df = pd.DataFrame(draws_data)

# Save to CSV (optional)
df.to_csv('euromillions_draws_beatlottery.csv', index=False)

# Display the first few rows of the DataFrame
print(df.head())

Extracting draws from the EuroMillions draw history page...
Found 52 rows in the table.
     Draw Date                  Main Balls Lucky Stars
0  01 Nov 2024  Fr, i , 07, 33, 38, 44, 50      02, 07
1  29 Oct 2024  Tu, e , 01, 02, 19, 30, 45      10, 12
2  25 Oct 2024  Fr, i , 04, 17, 20, 25, 45      08, 09
3  22 Oct 2024  Tu, e , 03, 13, 27, 30, 46      01, 02
4  18 Oct 2024  Fr, i , 04, 15, 23, 30, 39      01, 12
